# Type charts

By **Franklin Oliveira**

-----
This notebook contains all code necessary to make the "type" charts from `poliqueta` database. Here you'll find some basic data treatment and charts' code. 

Database: <font color='blue'>'IBUFRJ27.07.2020 - visualização.xlsx'</font> and <font color='blue'>'MNRJP27.07.2020 - visualização.xls'</font>.

In [2]:
import datetime
import numpy as np
import pandas as pd

from collections import defaultdict

# pacotes para visualização rápida
import seaborn as sns
import matplotlib.pyplot as plt

# pacote para visualização principal
import altair as alt

# habilitando renderizador para notebook
# alt.renderers.enable('notebook')
alt.renderers.enable('default')


# desabilitando limite de linhas
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

## Importing data...

In [3]:
NewTable = pd.read_csv('./data/merged_db.csv', sep=';', encoding='utf-8-sig')

In [4]:
# formatando a string NaN
NewTable['family'] = NewTable['family'].apply(lambda x: 'NaN' if x=='Nan' else x)

<br>

<font size=5>**Color Palette per Order**</font>

These images were used as inspiration (https://color.adobe.com/create/image)


<div class='row' style='padding-top:20px;'>
    <div class='col-md-6'>
        <img src="./src/img1.jpg" width='400px'>
    </div>
    <div class='col-md-6'>
        <img src="./src/img2.jpg" width='400px'>
    </div>
</div>

<br>

A partir das imagens acima, selecionamos cores (centróides) para criar a paleta de cores. Foram elas: 
<ul>
    <li style='color:#3CA67F'><b> #3CA67F </b># verde</li>
    <li style='color:#7A9FBF'><b> #7A9FBF </b># azul</li>
    <li style='color:#D94814'><b> #D94814 </b># laranja</li>
    <li style='color:#D96236'><b> #D96236 </b># laranja 2</li>
    <li style='color:#F2B999'><b> #F2B999 </b># 'cor de pele'</li>
    <li style='color:#A66C4B'><b> #A66C4B </b># marrom 1</li>
    <li style='color:#732C02'><b> #732C02 </b># marrom 2</li>
</ul>

A partir das cores "centróides", utilizamos a ferramenta Color Crafter para selecionar diferentes "shades" e auxiliar para categorização em diferentes grupos sugeridos pela equipe de Poliquetas do Museu Nacional. 

<ul>
    <li style='color:#3CA67F'><b> Verde: </b> ['#daffef', '#bbebd3', '#9adabc', '#77c8a5', '#57b791', '#3ca67f', '#2a9670', '#238762', '#257a56']</li>
    <li style='color:#7A9FBF'><b> Azul: </b> ['#e7e5df', '#ccd2d8', '#b2c0d0', '#96afc8', '#7a9fbf', '#5d90b6', '#3c81ae', '#0673a4', '#00669a']</li>
    <li style='color:#D94814'><b> laranja: </b> ['#ffbd84', '#ffaa74', '#ff9760', '#ff814b', '#fc6b36', '#eb5824', '#d94814', '#c83b03', '#b73000']</li>
    <li style='color:#D96236'><b> laranja 2: ['#ffeba9', '#ffd391', '#ffbb7b', '#fda468', '#f18e56', '#e57846', '#d96236', '#cc4d28', '#bf381b']</b> </li>
    <li style='color:#F2B999'><b> cor de pele: ['#ffe9c3', '#fbd0ad', '#f2b999', '#e8a287', '#dd8c76', '#d27666', '#c76158', '#bb4d4b', '#ae393e']</b> </li>
    <li style='color:#A66C4B'><b> marrom 1: ['#d9c6af', '#ccad96', '#c1977c', '#b48061', '#a66c4b', '#975b39', '#874c2c', '#774124', '#683720']</b> </li>
    <li style='color:#732C02'><b> marrom 2: ['#eebd93', '#dfa47a', '#d28d60', '#c37746', '#b4622f', '#a3501d', '#92420e', '#823606', '#732c02']</b> </li>
</ul>



**Colors  (antigas):** 

<ul>
    <li style='color:#41A681'><b> #41A681 </b># verde1</li>
    <li style='color:#3CA67F'><b> #3CA67F </b># verde2</li>
    <li style='color:#7ACAAB'><b> #7ACAAB </b># verde claro</li>
    <li style='color:#78a1a1'><b> #78a1a1 </b># azul</li>
    <li style='color:#8ABFB0'><b> #8ABFB0 </b># azul claro</li>
    <li style='color:#FFB27C'><b> #FFB27C </b># cor de pele clara</li>
    <li style='color:#F29877'><b> #F29877 </b># cor de pele</li>
    <li style='color:#ed845e'><b> #ed845e </b># laranja claro1</li>
    <li style='color:#D96236'><b> #D96236 </b># laranja claro2</li>
    <li style='color:#D95323'><b> #D95323 </b># laranja 1</li>
    <li style='color:#D94B18'><b> #D94B18 </b># laranja 2</li>
    <li style='color:#D9C2AD'><b> #D9C2AD </b># bege</li>
    <li style='color:#A66C4B'><b> #A66C4B </b># marrom claro</li>
    <li style='color:#86471B'><b> #86471B </b># marrom1</li>
    <li style='color:#732C02'><b> #732C02 </b># marrom2</li>
    <li style='color:#592202'><b> #592202 </b># marrom escuro1</li>
    <li style='color:#3D1806'><b> #3D1806 </b># marrom escuro2</li>
    <li style='color:#0D0D0D'><b> #0D0D0D </b># preto</li>
</ul>



In [5]:
# determinando cores de acordo com a planilha (2020.10.01 - IB e MN - Cores visualização.xlsx)
ordens = NewTable['order'].unique()
familias = NewTable['family'].unique()

# # o agrupamento é feito por famílias (ordem daquelas famílias deve assumir certa cor)
cores_ordem = {
    'Spionida':'#41A681',   # verde
    'Sabellida':'#7ACAAB',  # verde claro
    'Canalipalpata':'#78a1a1',  # azul
    'Amphinomida':'#8ABFB0',  # azul claro
    'Eunicida':'#A66C4B', # marrom claro
    'Phyllodocida':'#732C02', # marrom2
    'Terebellida':'#ed845e', # laranja claro1
    'Scolecida':'#D94B18', # laranja 2
    np.NAN:'#0D0D0D',  # preto
    
    # ordens não citadas na planilha:
    'Sipuncula':'#D9C2AD', # bege
    'Crassiclitellata':'#FFB27C', # cor de pele clara
    'Aspidosiphonida':'#F29877',  # cor de pele
    
}

# paleta de cores por família
cores_familia = {
    'Magelonidae':'#238762',    # verde escuro 
    'Oweniidae':'#3CA67F',      # verde (centroide)  
    'Chaetopteridae':'#77c8a5', # verde
    'Amphinomidae':'#bbebd3',   # verde claro
    'Lumbrineridae':'#e7e5df',  # azul claro 1
    'Dorvilleidae':'#b2c0d0',   # azul claro2
    'Oenonidae':'#7A9FBF',      # azul (centroide)
    'Eunicidae':'#3c81ae',      # azul
    'Onuphidae':'#00669a',      # azul escuro
    'Syllidae':'#ffbd84', 
    'Typhloscolecidae':'#ffaa74', 
    'Aphroditidae':'#ff9760', 
    'Acoetidae':'#ff814b', 
    'Chrysopetalidae':'#fc6b36', 
    'Eulepethidae':'#eb5824',
    'Lopadorrhynchidae':'#d94814',  # laranja (centroide)
    'Polynoidae':'#c83b03',
    'Nereididae':'#b73000',
    'Nephtyidae':'#f18e56',
    'Glyceridae':'#D96236',         # laranja 2 (centroide)
    'Tomopteridae':'#bf381b',
    'Serpulidae':'#fbd0ad',
    'Sabellidae':'#f2b999', # cor de pele (centroide)
    'Sabellariidae':'#e8a287',
    'Spionidae':'#d27666',
    'Ampharetidae':'#b48061',
    'Pectinariidae':'#a66c4b',  # marrom 1 (centroide),
    'Trichobranchidae':'#975b39',
    'Terebellidae':'#874c2c',
    'Cirratulidae':'#774124',
    'Flabelligeridae':'#683720',
    'Sternaspidae':'#eebd93',
    'Orbiniidae':'#dfa47a',
    'Opheliidae':'#d28d60',
    'Capitellidae':'#c37746',
    'Arenicolidae':'#b4622f',
    'Cossuridae':'#a3501d',
    'Scalibregmatidae':'#92420e',
    'Paraonidae':'#823606',
    'Maldanidae':'#732c02', # marrom 2 (centroide)
    'NaN':'#0D0D0D',  # preto
}

In [6]:
# paleta de cores ANTIGA

# ordens = NewTable['order'].unique()
# cores = [
#     '#8ABFB0',  # azul claro
#     '#41A681',  # verde
#     '#7ACAAB',  # verde claro
#     '#D9C2AD',  # bege
#     '#0D0D0D',  # preto
#     '#D96236',  # laranja
#     '#D94B18',  # laranja escuro
#     '#FFB27C',  # cor de pele clara
#     '#732C02',  # marrom
#     '#86471B',  # mostarda
    
#     # cores novas para Canalipalpata e Aspidosiphonida (a ordem é aleatória. Fixar depois)
#     '#592202',
#     '#D96236'
# ]

# cores_ordem = defaultdict()
# for j in range(len(ordens)):
#     ordem = ordens[j]
#     cores_ordem[ordem] = cores[j]
    
# cores_ordem = dict(cores_ordem)

<br>


## Graphs

---

### Types (*per year*) per genus

x: Species1, cor: Type Status1, size: counts

In [7]:
# p.s.: the large majority is non-type
NewTable['type'].value_counts()

Paratype    119
Holotype     35
Neotype       1
Name: type, dtype: int64

In [8]:
# subsetting
teste = NewTable[['min_depth','family','order', 'start_year', 'qualifier', 'catalog_number', 
                  'genus', 'species', 'type']].copy()

# grouping by type, year and order
temp = teste.groupby(['type','start_year', 'family']).count()['species'].reset_index().rename(columns={
    'species':'counts'
})

# p.s.: Cótipo and Topótipo are not types
temp = temp[(temp['type'] != 'Cótipo') & (temp['type'] != 'Topótipo')]

243 info. de tipos

### Gráf. de Tipos

In [11]:
tipo = alt.Chart(temp, height=150, title='Types per year').mark_circle().encode(
    x = alt.X('start_year:O', title='Sampling Year'),
    y = alt.Y('type:N', title= 'Type',
              sort=alt.EncodingSortField('counts', op='sum', order='descending')),
    color= alt.Color('family:N', title='Family',
                     scale=alt.Scale(domain=list(cores_familia.keys()), range=list(cores_familia.values())),
                     legend= alt.Legend(columns=2, symbolLimit=42)), 
    size= alt.Size('counts'),
    tooltip= [alt.Tooltip('type', title='type'),
              alt.Tooltip('start_year', title='start year'),
              alt.Tooltip('counts', title='counts')]
)

# tipo.save('./graphs/tipo/tipos_por_ano-colors_per_family.html')

tipo

alt.Chart(...)

## Types per Genus 

same graph as above, with gender on Y axis and colored by type

In [22]:
# subsetting
teste = NewTable[['min_depth','family','order', 'start_year', 'qualifier', 'catalog_number', 
                  'genus', 'species', 'type']].copy()

# grouping by type, year and order
temp = teste.groupby(['type','start_year', 'genus', 'family']).count()['order'].reset_index().rename(columns={
    'order':'counts'
})

# p.s.: Cótipo and Topótipo are not types
temp = temp[(temp['type'] != 'Cótipo') & (temp['type'] != 'Topótipo')]

In [23]:
cores_padrao = ['#e45756', '#4c78a8', '#f58518']
tipos = ['Holotype', 'Paratype', 'Neotype']

In [26]:
tipo = alt.Chart(temp, height=400, width= 400, title='Types per Genus').mark_point(filled=False).encode(
    x = alt.X('start_year:O', title='Sampling Year'),
    y = alt.Y('genus:N', title= 'Genus',
              sort=alt.EncodingSortField('counts', op='count', order='descending')),
    color= alt.Color('family:N', title='Family',
                    scale= alt.Scale(domain=list(cores_familia.keys()), range=list(cores_familia.values())),
                    legend= alt.Legend(columns=2, symbolLimit=42, symbolType= 'circle')), 
    size= alt.Size('counts', scale= alt.Scale(range=[10,500])),
    order= alt.Order('counts', sort='descending'),  # smaller points in front
    shape= alt.Shape('type:N', title='Type', 
                    scale= alt.Scale(domain=['Holotype', 'Neotype','Paratype'],
                                     range=['triangle', 'square', 'circle'])),
    opacity= alt.Opacity('type:N', scale= alt.Scale(domain=['Holotype', 'Neotype','Paratype'],
                                                     range=[1, 0.5, 1])),
    tooltip= [alt.Tooltip('type', title='type'),
              alt.Tooltip('start_year', title='start year'),
              alt.Tooltip('counts', title='counts')]
)

# tipo.save('./graphs/tipo/tipos_por_genero.html')

tipo

alt.Chart(...)

In [27]:
genus_order = list(temp.groupby(['genus']).min()['start_year'].reset_index().sort_values('start_year')['genus'])

In [30]:
tipo = alt.Chart(temp, height=400, width= 400, title='Types per Genus').mark_point(filled=False).encode(
    x = alt.X('start_year:O', title='Sampling Year'),
    y = alt.Y('genus:N', title= 'Genus',
              sort=genus_order),
    color= alt.Color('family:N', title='Family',
                    scale= alt.Scale(domain=list(cores_familia.keys()), range=list(cores_familia.values())),
                    legend= alt.Legend(columns=2, symbolLimit=42)), 
    size= alt.Size('counts', scale=alt.Scale(range=[10,500])),
    order= alt.Order('counts', sort='descending'),  # smaller points in front
    shape= alt.Shape('type:N', title='Type', 
                    scale= alt.Scale(domain=['Holotype', 'Neotype','Paratype'],
                                     range=['triangle', 'square', 'circle'])),
    opacity= alt.Opacity('type:N', scale= alt.Scale(domain=['Holotype', 'Neotype','Paratype'],
                                                     range=[1, 0.5, 1])),
    tooltip= [alt.Tooltip('type', title='type'),
              alt.Tooltip('start_year', title='start year'),
              alt.Tooltip('counts', title='counts')]
)

# tipo.save('./graphs/tipo/tipos_por_genero-primeiro_ano.html')

tipo

alt.Chart(...)

## Types per family

In [17]:
# subsetting
teste = NewTable[['min_depth','family','order', 'start_year', 'qualifier', 'catalog_number', 
                  'genus', 'species', 'type']].copy()

# grouping by type, year and order
temp = teste.groupby(['type','start_year', 'family', 'order']).count()['genus'].reset_index().rename(columns={
    'genus':'counts'
})

# p.s.: Cótipo and Topótipo are not types
temp = temp[(temp['type'] != 'Cótipo') & (temp['type'] != 'Topótipo')]

In [18]:
cores_padrao = ['#e45756', '#4c78a8', '#f58518']
tipos = ['Holotype', 'Paratype', 'Neotype']

In [21]:
tipo = alt.Chart(temp, height=400, width=400, title='Types per Family').mark_point(filled=False).encode(
    x = alt.X('start_year:O', title='Sampling Year'),
    y = alt.Y('family:N', title= 'Family',
              sort=alt.EncodingSortField('counts', op='sum', order='descending')),
    color= alt.Color('family:N', title='Family',
                    scale= alt.Scale(domain=list(cores_familia.keys()), range=list(cores_familia.values())),
                    legend= alt.Legend(columns=2, symbolLimit=42)), 
    size= alt.Size('counts', scale=alt.Scale(range=[10,500])),
    order= alt.Order('counts', sort='descending'),  # smaller points in front
    shape= alt.Shape('type:N', title='Type', 
                    scale= alt.Scale(domain=['Holotype', 'Neotype','Paratype'],
                                     range=['triangle', 'square', 'circle'])),
    opacity= alt.Opacity('type:N', scale= alt.Scale(domain=['Holotype', 'Neotype','Paratype'],
                                                     range=[1, 0.5, 1])),
    tooltip= [alt.Tooltip('type', title='type'),
              alt.Tooltip('start_year', title='start year'),
              alt.Tooltip('counts', title='counts')]
)

# tipo.save('./graphs/tipo/tipos_por_familia.html')

tipo

alt.Chart(...)

<br>

**The end!**

-----